Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

IOError: [Errno 2] No such file or directory: 'notMNIST.pickle'

Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [46]:
def accuracy(predictions, labels, print_entries=False):
  if (print_entries):
    print(predictions[0,])
    print(labels[0,])
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [7]:
# Regularized Logistic Regression
batch_size = 128
C = 0.02

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)+C*tf.nn.l2_loss(weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 78.272156
Minibatch accuracy: 8.6%
Validation accuracy: 10.2%
Minibatch loss at step 500: 0.834336
Minibatch accuracy: 78.1%
Validation accuracy: 81.4%
Minibatch loss at step 1000: 0.626871
Minibatch accuracy: 85.2%
Validation accuracy: 80.8%
Minibatch loss at step 1500: 1.005861
Minibatch accuracy: 77.3%
Validation accuracy: 81.0%
Minibatch loss at step 2000: 0.801588
Minibatch accuracy: 82.8%
Validation accuracy: 81.6%
Minibatch loss at step 2500: 0.795578
Minibatch accuracy: 82.8%
Validation accuracy: 81.7%
Minibatch loss at step 3000: 0.955648
Minibatch accuracy: 76.6%
Validation accuracy: 80.1%
Test accuracy: 86.7%


In [9]:
# Regularized Neural Networks with 1 hidden layer
batch_size = 128
num_hidden = 1024
C = 0.02

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden]))
  biases = tf.Variable(tf.zeros([num_hidden]))
    
  weights_hidden = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
  biases_hidden = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  #logits = tf.matmul(tf_train_dataset, weights) + biases
  #loss = tf.reduce_mean(
  #  tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights)+biases), weights_hidden)+biases_hidden
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
                        +C*tf.nn.l2_loss(weights)
                        +C*tf.nn.l2_loss(weights_hidden))
  # should we regularize bias???????????????????????????
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases), weights_hidden)+biases_hidden)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases), weights_hidden)+biases_hidden)

In [10]:
num_steps = 3001

'''def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])'''

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 6619.140625
Minibatch accuracy: 12.5%
Validation accuracy: 28.6%
Minibatch loss at step 500: 1.202575
Minibatch accuracy: 81.2%
Validation accuracy: 83.0%
Minibatch loss at step 1000: 0.675161
Minibatch accuracy: 89.8%
Validation accuracy: 82.0%
Minibatch loss at step 1500: 0.967635
Minibatch accuracy: 79.7%
Validation accuracy: 82.8%
Minibatch loss at step 2000: 0.801126
Minibatch accuracy: 82.8%
Validation accuracy: 82.9%
Minibatch loss at step 2500: 0.836307
Minibatch accuracy: 85.2%
Validation accuracy: 82.6%
Minibatch loss at step 3000: 0.948253
Minibatch accuracy: 81.2%
Validation accuracy: 82.5%
Test accuracy: 88.7%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [15]:
# Regularized Neural Networks with 1 hidden layer
batch_size = 128
num_hidden = 1024
C = 0
keep_prob = 1

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden]))
  biases = tf.Variable(tf.zeros([num_hidden]))
    
  weights_hidden = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
  biases_hidden = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  #logits = tf.matmul(tf_train_dataset, weights) + biases
  #loss = tf.reduce_mean(
  #  tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  logits = tf.matmul(tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights)+biases),keep_prob=keep_prob), weights_hidden)+biases_hidden
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
                        +C*tf.nn.l2_loss(weights)
                        +C*tf.nn.l2_loss(weights_hidden))
  # should we regularize bias???????????????????????????
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases), weights_hidden)+biases_hidden)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases), weights_hidden)+biases_hidden)

In [16]:
num_steps = 3001

'''def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])'''

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (200)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 311.642853
Minibatch accuracy: 11.7%
Validation accuracy: 32.1%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.8%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.8%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.8%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.8%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.8%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.8%
Test accuracy: 81.3%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [35]:
# Regularized Neural Networks with 1 hidden layer
batch_size = 128
num_hidden = 1024
C = 0
keep_prob = 0.5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden]))
  biases = tf.Variable(tf.zeros([num_hidden]))
    
  weights_hidden = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
  biases_hidden = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  #logits = tf.matmul(tf_train_dataset, weights) + biases
  #loss = tf.reduce_mean(
  #  tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  logits = tf.matmul(tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights)+biases),keep_prob=keep_prob), weights_hidden)+biases_hidden
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
                        +C*tf.nn.l2_loss(weights)
                        +C*tf.nn.l2_loss(weights_hidden))
  # should we regularize bias???????????????????????????
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases), weights_hidden)+biases_hidden)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases), weights_hidden)+biases_hidden)

In [36]:
num_steps = 3001

'''def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])'''

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % 200
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 436.691589
Minibatch accuracy: 15.6%
Validation accuracy: 16.4%
Minibatch loss at step 500: 50.393509
Minibatch accuracy: 68.0%
Validation accuracy: 79.0%
Minibatch loss at step 1000: 6.514162
Minibatch accuracy: 85.2%
Validation accuracy: 80.0%
Minibatch loss at step 1500: 16.326124
Minibatch accuracy: 68.0%
Validation accuracy: 79.5%
Minibatch loss at step 2000: 4.543086
Minibatch accuracy: 73.4%
Validation accuracy: 80.0%
Minibatch loss at step 2500: 5.992505
Minibatch accuracy: 78.9%
Validation accuracy: 80.0%
Minibatch loss at step 3000: 6.767577
Minibatch accuracy: 72.7%
Validation accuracy: 80.5%
Test accuracy: 87.2%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [57]:
 def logits_func(weights, biases, keep_prob, first_layer_neuron, first_layer_value):
    previous_neuron = first_layer_neuron
    previous_value = first_layer_value
    for hidden_no in range(len(weights)):
        #print(previous_value.shape)
        #print(weights[hidden_no].shape)
        if (hidden_no==(len(weights)-1)): # the output layer            
            previous_value = tf.matmul(previous_value, weights[hidden_no])+biases[hidden_no]
        else:
            previous_value = tf.nn.dropout(tf.nn.relu(
            tf.matmul(previous_value, weights[hidden_no])+biases[hidden_no]),keep_prob=keep_prob)
    return(previous_value)

In [63]:
# using dropout as regularization
# using multiple layers
# using learning rate decay

# set parameters
batch_size = 128
num_hidden = np.array([1024,10]) # include the output layer
keep_prob = 0.5
num_steps = 3001
decay_steps = 300
decay_rate = 0.96

#print(num_hidden.shape[0])
#print(range(num_hidden.shape[0]))

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = [] # list of tf variables
  biases = [] # list of tf variables
  previous_neuron = image_size * image_size  
  for hidden_no in range(num_hidden.shape[0]):    
      weights.append(tf.Variable(
        tf.truncated_normal([previous_neuron, num_hidden[hidden_no]])))
      previous_neuron = num_hidden[hidden_no]
      biases.append(tf.Variable(tf.zeros([num_hidden[hidden_no]])))      
      
  # Training computation.      
  logits = logits_func(
        weights=weights, biases=biases, keep_prob=keep_prob, 
        first_layer_neuron=image_size*image_size, first_layer_value=tf_train_dataset)  

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))  
  
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, decay_steps=decay_steps, decay_rate=decay_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)  
    
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    logits_func(
        weights=weights, biases=biases, keep_prob=1, 
        first_layer_neuron=image_size*image_size, first_layer_value=tf_valid_dataset))
  test_prediction = tf.nn.softmax(
    logits_func(
        weights=weights, biases=biases, keep_prob=1, 
        first_layer_neuron=image_size*image_size, first_layer_value=tf_test_dataset))

In [64]:
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels, False))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels, False))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels, False))

Initialized
Minibatch loss at step 0: 519.910950
Minibatch accuracy: 7.0%
Validation accuracy: 27.4%
Minibatch loss at step 500: 33.629562
Minibatch accuracy: 70.3%
Validation accuracy: 80.7%
Minibatch loss at step 1000: 14.629493
Minibatch accuracy: 78.9%
Validation accuracy: 78.7%
Minibatch loss at step 1500: 14.558258
Minibatch accuracy: 72.7%
Validation accuracy: 80.1%
Minibatch loss at step 2000: 4.531221
Minibatch accuracy: 75.8%
Validation accuracy: 79.9%
Minibatch loss at step 2500: 4.685804
Minibatch accuracy: 75.8%
Validation accuracy: 80.1%
Minibatch loss at step 3000: 6.507464
Minibatch accuracy: 74.2%
Validation accuracy: 80.0%
Test accuracy: 87.4%


In [ ]:
aa